In [1]:
#Initial environement of computations


import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests

from kanon.units import Sexagesimal, Historical, IntegerAndSexagesimal, Temporal
from kanon.calendars import*

Cal=julian_ad = Calendar.registry["Julian A.D."]

number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

# Historical target table
Hist_table=HTable.read(576, with_units=False)
Hist_table


"Years, opposition",Entries
IntegerAndSexagesimal,Historical
482495 ;,"10s 01 ; 48,02,23"
491261 ;,"10s 06 ; 29,47,57"
500027 ;,"10s 11 ; 11,33,31"
508793 ;,"10s 15 ; 53,19,05"
517559 ;,"09s 21 ; 28,40,28"
526325 ;,"09s 26 ; 10,26,02"
535091 ;,"10s 00 ; 52,11,36"
543857 ;,"10s 05 ; 33,57,11"
552623 ;,"10s 10 ; 15,42,45"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(13):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)

In [4]:
#mean motion parameter from the Parisian Alfonsine Tables
daily_mean_elong=Historical(Sexagesimal("12;11,26,41,37,51,50,39"),7)
radice_elong=Historical(Sexagesimal("3,24;25,49,46,11"),4)
daily_mean_sun=Historical(Sexagesimal("0;59,8,19,37,19,13,56"),8)
radice_sun=Historical(Sexagesimal("4,38;21,0,30,28"),4)
delta_Paris_Toledo=Temporal("0;0,48")
REVO=Historical(Sexagesimal("6,0;0,0"),7)
epoch= Date(Cal, (1, 1, 1))

for i in range(13):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=7):

        #conversion of date to DN from calendar epoch
        days=int(Hist_table[i][0])-365

        #computation of the elongation at date
        elong=(days*daily_mean_elong+radice_elong)%REVO

        #computation of time to conjonction NOT including delta between Paris and Toledo and conversion in DN since epoch
        time=(REVO-elong)/daily_mean_elong
        time=Temporal(time,7)+REVO/(2*daily_mean_elong)
        if time>30:
            time=time-REVO/daily_mean_elong
        else:
            time=time
        time=time.decimal
        time=float(time)+days
        
        #computation of the mean sun (and moon) for that time
        mean_sun_moon=(time*daily_mean_sun+radice_sun)%REVO
        
        
        #filling the Test and Delta tables with the final precision
        with set_precision(tmode=TruncatureMode.ROUND, pmode=3):
            Test_recomp[i][1]=1*mean_sun_moon
            Delta_recomp[i][1]=60*60*60*(Hist_table[i][1]-Test_recomp[i][1])
Test_recomp

"Years, opposition",Entries
IntegerAndSexagesimal,Historical
482495 ;,"10s 01 ; 48,02,23"
491261 ;,"10s 06 ; 29,47,57"
500027 ;,"10s 11 ; 11,33,31"
508793 ;,"10s 15 ; 53,19,05"
517559 ;,"09s 21 ; 28,40,28"
526325 ;,"09s 26 ; 10,26,02"
535091 ;,"10s 00 ; 52,11,36"
543857 ;,"10s 05 ; 33,57,11"
552623 ;,"10s 10 ; 15,42,45"


In [5]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(0.07692307692307693, nan)

In [6]:
B.astype(int).style.background_gradient(axis=None)

,Entries
"Years, opposition",
482495 ;,0
491261 ;,0
500027 ;,0
508793 ;,0
517559 ;,0
526325 ;,0
535091 ;,0
543857 ;,0
552623 ;,0


In [7]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Mean syzygy mean sun years opposition.csv')
B.to_csv('Mean syzygy mean sun years opposition.zip', index=False, compression=compression_opts)